<a href="https://colab.research.google.com/github/Ghazaleh99/language_ditector_KfoldCrossValidation/blob/main/ANN_HW1_150901012_Hamzeh_3_Cross_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#تمرین یک درس شبکه های عصبی
#5-fold Cross-Validation
###**سوال سوم**

غزاله حمزه

##قسمت ج


###Step 1 - Import the library

In [ ]:
import pandas as pd
import random
import math

###Step 2 - Setting up the Data

In [ ]:
langD_file_path = 'ANN-HW1-Data-LangID.xlsx'
lang_data = pd.read_excel(langD_file_path)
lang_data.columns

Index(['جملات', 'زبان'], dtype='object')

###Step 3 - Splitting the Data

In [ ]:
k_fold = 5
data_shape = lang_data.shape
print('Data shape is',data_shape)
validation_size = int(data_shape[0]) // k_fold 
print('valdation size', validation_size)

Data shape is (150, 2)
valdation size 30


####taking one sentence from evrey language to make a good k_fold dataset!

In [ ]:
train_size = (k_fold-1)*validation_size
test_size = validation_size
data_size = int(data_shape[0])
data_set = []
rand_list0 = random.sample(range(50), 50)
rand_list1 = random.sample(range(50,100), 50)
rand_list2 = random.sample(range(100,150), 50)

for i in range(data_size//3):
  data_set.append(lang_data['جملات'][rand_list0[i]]) #arab
  data_set.append(lang_data['جملات'][rand_list1[i]]) #persain
  data_set.append(lang_data['جملات'][rand_list2[i]]) #kor

print(data_set[0])
print(data_set[1])
print(data_set[2])

الحكومة الإسرائيلية أقامت وحدات سكنية لاستيعاب المستوطنين
اتسز نتوانست به توسعه قلمرو خوارزمشاهیان کمک چندانی بکند
لە بیرمان بێ کە کاری زمانەوان ناساندنە ، نەوەک دانان


In [ ]:
def lang_data_splitter(data, d_size, split, lang):
  new_list = []
  for i in range(d_size):
    tmp = data[i].replace(' ','')
    tmp = tmp.replace('\xa0','')
    tmp = tmp.replace('\u200c','')
    tmp = tmp.replace('،','')
    if i%split == lang:
      new_list.append(tmp)
  return new_list

#### bag of words

In [ ]:
def bagAlphabet(arb_train_set, per_train_set, kor_train_set):
  bag_alphabet = []
  for i in arb_train_set:
    for j in i:
      if j not in bag_alphabet:
        bag_alphabet.append(j)

  for i in per_train_set:
    for j in i:
      if j not in bag_alphabet:
        bag_alphabet.append(j)

  for i in kor_train_set:
    for j in i:
      if j not in bag_alphabet:
        bag_alphabet.append(j)
  # print(bag_alphabet)
  return bag_alphabet

##Step 4 - Train

#####Computing the normal CF

In [ ]:
def normalCF(charFrequency_lang, bag_alphabet, train_size):
  charFrequency, lang_normalCF = dict_temp(bag_alphabet)
  for i in charFrequency.keys():
    for j in range(train_size):
      ncf = charFrequency_lang[i][j]/len(charFrequency_lang[i])
      lang_normalCF[i].append(ncf)

  lang_sumNFC = charFrequency.copy()
  for i in charFrequency.keys():
    tmp = 0
    for j in range(train_size):
      tmp += lang_normalCF[i][j]
    lang_sumNFC[i] = tmp/train_size
  # print(lang_sumNFC)
  
  return lang_sumNFC

###Step 4 - Train with Character Frequency (CF)

In [ ]:
def train(train_set, train_size):
  arb_train_set = lang_data_splitter(train_set, train_size, 3, 0)
  per_train_set = lang_data_splitter(train_set, train_size, 3, 1)
  kor_train_set = lang_data_splitter(train_set, train_size, 3, 2)
  train_size = len(arb_train_set)
  # print(per_train_set)

  bag_alphabet = bagAlphabet(arb_train_set, per_train_set, kor_train_set)

  charFrequency_arb = charFC(arb_train_set, bag_alphabet)
  charFrequency_per = charFC(per_train_set, bag_alphabet)
  charFrequency_kor = charFC(kor_train_set, bag_alphabet)
  # print(len(charFrequency_kor['ڤ']))
  # print(len(charFrequency_per['ڤ']))
  # print(len(charFrequency_arb['ڤ']))

  arb_sumNFC = normalCF(charFrequency_arb, bag_alphabet, train_size)
  per_sumNFC = normalCF(charFrequency_per, bag_alphabet, train_size)
  kor_sumNFC = normalCF(charFrequency_kor, bag_alphabet, train_size)
  
  return arb_sumNFC, per_sumNFC, kor_sumNFC, bag_alphabet

###Character Frequency (CF)

In [ ]:
def dict_temp(bag_alphabet):
  charFrequency_lang = {}
  charFrequency = {}
  for i in bag_alphabet:
    if i not in charFrequency_lang.keys():
      charFrequency_lang[i] = [] #list of repeatation for evrey sentence
      charFrequency[i] = int(0) 
  # print(len(charFrequency))
  return charFrequency, charFrequency_lang

In [ ]:
def charFC(t_data, bag_alphabet):
  charFrequency, charFrequency_lang = dict_temp(bag_alphabet)
  for i in t_data:
    i = i.replace(' ','')
    i = i.replace('\xa0','')
    i = i.replace('\u200c','')
    i = i.replace('،','')
    # print(i)
    for j in i:
      if j in charFrequency.keys():
        charFrequency[j] += int(1)
    for k in charFrequency.keys():
      charFrequency_lang[k].append(charFrequency[k])
  return charFrequency_lang

##Step 5 - Test

In [ ]:
def ncf_testing(charFrequency_lang, bag_alphabet, test_size):
  charFrequency, lang_normalCF = dict_temp(bag_alphabet)
  for i in charFrequency.keys():
    for j in range(test_size):
      ncf = charFrequency_lang[i][j]/len(charFrequency_lang[i])
      lang_normalCF[i].append(ncf)
  return lang_normalCF

In [ ]:
def predict(ncf_test, ncf_arb, ncf_per, ncf_kor):
  arb_Euclidean, per_Euclidean, kor_Euclidean = 0, 0, 0
  for i in ncf_test.keys():
    # print(ncf_arb[i],ncf_test[i])
    tmp = pow(ncf_arb[i] - ncf_test[i], 2)
    arb_Euclidean += tmp
    tmp = pow(ncf_per[i] - ncf_test[i], 2)
    per_Euclidean += tmp
    tmp = pow(ncf_kor[i] - ncf_test[i], 2)
    kor_Euclidean += tmp

  arb_Euclidean = math.sqrt(arb_Euclidean)
  per_Euclidean = math.sqrt(per_Euclidean)
  kor_Euclidean = math.sqrt(kor_Euclidean)
  # print(arb_Euclidean, per_Euclidean, kor_Euclidean)
  tmp = min(arb_Euclidean, per_Euclidean, kor_Euclidean)
  if tmp == arb_Euclidean:
    result = 'Arabic'
  elif tmp == per_Euclidean:
    result = 'Persain'
  elif tmp == kor_Euclidean:
    result = 'Kordish'

  return result

In [ ]:
def resultFunc(test_data, bag_alphabet, arb_sumNFC, per_sumNFC, kor_sumNFC):
  charFrequency, lang_normalCF = dict_temp(bag_alphabet)
  charFrequency_tmp = charFC(test_data, bag_alphabet)
  ncf_test_tmp = ncf_testing(charFrequency_tmp, bag_alphabet, len(test_data))
  
  test_result_list = []
  for i in range(len(test_data)):
    charFrequencytest = charFrequency.copy()
    for j in ncf_test_tmp.keys():
      charFrequencytest[j] = ncf_test_tmp[j][i]
    tmp = predict(charFrequencytest, arb_sumNFC, per_sumNFC, kor_sumNFC)
    test_result_list.append(tmp)
  return test_result_list

###Step 5 - Test with with Character Frequency (CF) 

In [ ]:
def test(test_set, test_size, bag_alphabet, arb_sumNFC, per_sumNFC, kor_sumNFC):
  arb_test_set = lang_data_splitter(test_set, test_size, 3, 0)
  per_test_set = lang_data_splitter(test_set, test_size, 3, 1)
  kor_test_set = lang_data_splitter(test_set, test_size, 3, 2)

  test_result_arab = resultFunc(arb_test_set, bag_alphabet, arb_sumNFC, per_sumNFC, kor_sumNFC)
  test_result_per = resultFunc(per_test_set, bag_alphabet, arb_sumNFC, per_sumNFC, kor_sumNFC)
  test_result_kor = resultFunc(kor_test_set, bag_alphabet, arb_sumNFC, per_sumNFC, kor_sumNFC)

  return test_result_arab, test_result_per, test_result_kor

##Step6 - Accuracy / F1-measure / Recall / Precision

####Accuracy = TN + TP / Total

In [ ]:
def accuracy(test_result_arab, test_result_per, test_result_kor):
  total = 30
  Tpos1, Tpos2, Tpos3 = 0, 0, 0
  Tneg1, Tneg2, Tneg3 = 0, 0, 0

  for i in test_result_arab:
    if i == 'Arabic':
      Tpos1 += 1
      Tneg3 += 1
      Tneg2 += 1
    elif i == 'Persain':
      Tneg3 += 1
    else:
      Tneg2 += 1

  for i in test_result_per:
    if i == 'Persain':
      Tpos2 += 1
      Tneg3 += 1
      Tneg1 += 1
    elif i == 'Arabic':
      Tneg3 += 1
    else:
      Tneg1 += 1

  for i in test_result_kor:
    if i == 'Kordish':
      Tpos3 += 1
      Tneg1 += 1
      Tneg2 += 1
    elif i == 'Persain':
      Tneg1 += 1
    else:
      Tneg2 += 1

  accuracy_arab = (Tpos1+Tneg1)/total
  accuracy_per = (Tpos2+Tneg2)/total
  accuracy_kor = (Tpos3+Tneg3)/total

  print('Accuracy:')
  print('Accuracy for Arabic is', accuracy_arab)
  print('Accuracy for Persain is', accuracy_per)
  print('Accuracy for Kordish is', accuracy_kor)

  return Tpos1, Tpos2, Tpos3, accuracy_arab, accuracy_per, accuracy_kor

####Recall = TP/TP+FN


In [ ]:
def recall(Tpos1, Tpos2, Tpos3):
  recall_arab = Tpos1/10 #10 - FN = TP
  recall_per = Tpos2/10
  recall_kor = Tpos3/10
  print('Recall:')
  print('Recall for Arabic is', recall_arab)
  print('Recall for Persain is', recall_per)
  print('Recall for Kordish is', recall_kor)

  return recall_arab, recall_per, recall_kor

####Precision = TP / TP + FP

In [ ]:
def percision(test_result_arab, test_result_per, test_result_kor):
  total = 30
  Tpos1, Tpos2, Tpos3 = 0, 0, 0
  fpos1, fpos2, fpos3 = 0, 0, 0

  for i in test_result_arab:
    if i == 'Arabic':
      Tpos1 += 1
    elif i == 'Persain':
      fpos2 += 1
    else:
      fpos3 += 1

  for i in test_result_per:
    if i == 'Persain':
      Tpos2 += 1
    elif i == 'Arabic':
      fpos1 += 1
    else:
      fpos3 += 1

  for i in test_result_kor:
    if i == 'Kordish':
      Tpos3 += 1
    elif i == 'Persain':
      fpos2 += 1
    else:
      fpos1 += 1

  # print('Tpos1+fpos1', Tpos1+fpos1)
  precision_arab = Tpos1/(Tpos1+fpos1)
  precision_per = Tpos2/(Tpos2+fpos2)
  precision_kor = Tpos3/(Tpos3+fpos3)

  print('Precision:')
  print('Precision for Arabic is', precision_arab)
  print('Precision for Persain is', precision_per)
  print('Precision for Kordish is', precision_kor)

  return precision_arab, precision_per, precision_kor

####F1-Measure = 2 * precision * recall / ( precision + recall )

In [ ]:
def fMeasure(lang, precision_lang, recall_lang):
  FMeasure_lang = 2 * precision_lang * recall_lang / (precision_lang + recall_lang)
  print('F1-Measure for', lang, 'is', FMeasure_lang)
  return FMeasure_lang

##Step 7 - Evaluate  

In [ ]:
def evaluate(test_result_arab, test_result_per, test_result_kor):
  print('___________________________________________________________________')
  
  eval_dic = {}

  Tpos1, Tpos2, Tpos3, accuracy_arab, accuracy_per, accuracy_kor = accuracy(test_result_arab, test_result_per, test_result_kor)

  try:
    recall_arab, recall_per, recall_kor = recall(Tpos1, Tpos2, Tpos3)
  except:
    print('RECALL:float division by zero')

  try:
    precision_arab, precision_per, precision_kor = percision(test_result_arab, test_result_per, test_result_kor)
  except:
    print('PRECISION:float division by zero')

  print('F1-Measure:')
  try:
    FMeasure_arab = fMeasure('Arabic', precision_arab, recall_arab)
  except:
    print('Arabic:float division by zero')
  try:
    FMeasure_per = fMeasure('Persain', precision_per, recall_per)
  except:
    print('Persain:float division by zero')
  try:
    FMeasure_kor = fMeasure('Kordish', precision_kor, recall_kor)
  except:
    print('Kordish:float division by zero')

  eval_dic['arb'] = [accuracy_arab, recall_arab, precision_arab, FMeasure_arab]
  eval_dic['per'] = [accuracy_per, recall_per, precision_per, FMeasure_per]
  eval_dic['kor'] = [accuracy_kor, recall_kor, precision_kor, FMeasure_kor]

  return eval_dic

#RUN THE WHOLE CODE

In [ ]:
acc1, pre1, rec1, fmsr1 = 0, 0, 0, 0 #arabic
acc2, pre2, rec2, fmsr2 = 0, 0, 0, 0 #persain
acc3, pre3, rec3, fmsr3 = 0, 0, 0, 0 #kordish

for i in range(k_fold):
  test_set = []
  train_set = []
  test_set = data_set[i*test_size:i*test_size+30]
  train_set = data_set[:i*test_size]
  train_set += data_set[i*test_size+30:]
  # print(len(train_set))

  arb_sumNFC, per_sumNFC, kor_sumNFC, bag_alphabet = train(train_set, train_size)


  test_result_arab, test_result_per, test_result_kor = test(test_set, test_size, bag_alphabet, arb_sumNFC, per_sumNFC, kor_sumNFC)
  print(f'________________________FOLD {i}_________________________________')
  evaluate_dic = evaluate(test_result_arab, test_result_per, test_result_kor)

  acc1 += evaluate_dic['arb'][0]
  acc2 += evaluate_dic['per'][0]
  acc3 += evaluate_dic['kor'][0]

  rec1 += evaluate_dic['arb'][1]
  rec2 += evaluate_dic['per'][1]
  rec3 += evaluate_dic['kor'][1]

  pre1 += evaluate_dic['arb'][2]
  pre2 += evaluate_dic['per'][2]
  pre3 += evaluate_dic['kor'][2]

  fmsr1 += evaluate_dic['arb'][3]
  fmsr2 += evaluate_dic['per'][3]
  fmsr3 += evaluate_dic['kor'][3]

print('_____________________________________________________________________')
print('______________________Version3_CF & K fold___________________________')
print('______________________________ALL____________________________________')

print('Accuracy:')
print('Accuracy for Arabic is', acc1/k_fold)
print('Accuracy for Persain is', acc2/k_fold)
print('Accuracy for Kordish is', acc3/k_fold)

print('Recall:')
print('Recall for Arabic is', rec1/k_fold)
print('Recall for Persain is', rec2/k_fold)
print('Recall for Kordish is', rec3/k_fold)

print('Precision:')
print('Precision for Arabic is', pre1/k_fold)
print('Precision for Persain is', pre2/k_fold)
print('Precision for Kordish is', pre3/k_fold)

print('F1-Measure:')
print('F1-Measure for Arabic is', fmsr1/k_fold)
print('F1-Measure for Persain is', fmsr2/k_fold)
print('F1-Measure for Kordish is', fmsr3/k_fold)

________________________FOLD 0_________________________________
_____________________________________________________________________
Accuracy:
Accuracy for Arabic is 1.0
Accuracy for Persain is 0.9333333333333333
Accuracy for Kordish is 0.9333333333333333
Recall:
Recall for Arabic is 1.0
Recall for Persain is 1.0
Recall for Kordish is 0.8
Precision:
Precision for Arabic is 1.0
Precision for Persain is 0.8333333333333334
Precision for Kordish is 1.0
F1-Measure:
F1-Measure for Arabic is 1.0
F1-Measure for Persain is 0.9090909090909091
F1-Measure for Kordish is 0.888888888888889
________________________FOLD 1_________________________________
_____________________________________________________________________
Accuracy:
Accuracy for Arabic is 1.0
Accuracy for Persain is 0.9
Accuracy for Kordish is 0.9
Recall:
Recall for Arabic is 1.0
Recall for Persain is 1.0
Recall for Kordish is 0.7
Precision:
Precision for Arabic is 1.0
Precision for Persain is 0.7692307692307693
Precision for Kordish